In [2]:
import pandas as pd
import numpy as np
import urllib.request
import zipfile
import random
import itertools
import math
import shapefile
from shapely.geometry import Polygon
from descartes.patch import PolygonPatch
import matplotlib as mpl
import matplotlib.pyplot as plt
plt.style.use('ggplot')
%matplotlib inline

In [3]:
#GREEN_TAXI_DATA_2018_JUNE

df_g = pd.read_csv("/Users/ranjani/taxidata/green_tripdata_2018-06.csv")
df_g.head()

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type
0,2,2018-06-01 00:33:55,2018-06-01 00:36:13,N,1,66,33,5,0.51,4.0,0.5,0.5,0.70,0.0,NaN,0.3,6.00,1,1
1,2,2018-06-01 00:40:36,2018-06-01 00:49:46,N,1,25,49,5,1.97,9.0,0.5,0.5,2.06,0.0,NaN,0.3,12.36,1,1
2,2,2018-06-01 00:57:12,2018-06-01 01:02:58,N,1,61,49,5,1.40,6.5,0.5,0.5,0.00,0.0,NaN,0.3,7.80,2,1
3,2,2018-06-01 00:10:13,2018-06-01 00:16:27,N,1,49,97,1,1.36,7.0,0.5,0.5,0.00,0.0,NaN,0.3,8.30,2,1
4,1,2018-06-01 00:32:08,2018-06-01 00:52:06,N,1,75,127,1,7.90,24.0,0.5,0.5,6.30,0.0,NaN,0.3,31.60,1,1


In [4]:
df_g.count()

VendorID                 739373
lpep_pickup_datetime     739373
lpep_dropoff_datetime    739373
store_and_fwd_flag       739373
RatecodeID               739373
PULocationID             739373
DOLocationID             739373
passenger_count          739373
trip_distance            739373
fare_amount              739373
extra                    739373
mta_tax                  739373
tip_amount               739373
tolls_amount             739373
ehail_fee                     0
improvement_surcharge    739373
total_amount             739373
payment_type             739373
trip_type                739373
dtype: int64

In [7]:
df_g['passenger_count'].isnull().sum()

0

In [5]:
urllib.request.urlretrieve("https://s3.amazonaws.com/nyc-tlc/misc/taxi_zones.zip", "taxi_zones.zip")
with zipfile.ZipFile("taxi_zones.zip","r") as zip_ref:
    zip_ref.extractall("./shape")

In [6]:
def get_lat_lon(sf):
    content = []
    for sr in sf.shapeRecords():
        shape = sr.shape
        rec = sr.record
        loc_id = rec[shp_dic['LocationID']]
        
        x = (shape.bbox[0]+shape.bbox[2])/2
        y = (shape.bbox[1]+shape.bbox[3])/2
        
        content.append((loc_id, x, y))
    return pd.DataFrame(content, columns=["LocationID", "longitude", "latitude"])

In [7]:
sf = shapefile.Reader("shape/taxi_zones.shp")
fields_name = [field[0] for field in sf.fields[1:]]
shp_dic = dict(zip(fields_name, list(range(len(fields_name)))))
attributes = sf.records()
shp_attr = [dict(zip(fields_name, attr)) for attr in attributes]

df_loc = pd.DataFrame(shp_attr).join(get_lat_lon(sf).set_index("LocationID"), on="LocationID")
df_loc.head()

,LocationID,OBJECTID,Shape_Area,Shape_Leng,borough,zone,longitude,latitude
0,1,1,0.000782,0.116357,EWR,Newark Airport,9.366817e+05,190522.130278
1,2,2,0.004866,0.433470,Queens,Jamaica Bay,1.033536e+06,161853.982300
2,3,3,0.000314,0.084341,Bronx,Allerton/Pelham Gardens,1.027136e+06,254730.010849
3,4,4,0.000112,0.043567,Manhattan,Alphabet City,9.904240e+05,203100.040432
4,5,5,0.000498,0.092146,Staten Island,Arden Heights,9.321332e+05,139954.541936


In [10]:
df_g = df_g.join(df_loc.set_index('LocationID'),on=['PULocationID'])
df_g.drop(['OBJECTID', 'Shape_Leng', 'Shape_Area', 'zone', 'borough'], axis=1, inplace=True)
df_g.rename(columns={'longitude': 'pickup_longitude', 'latitude': 'pickup_latitude'}, inplace=True)

In [11]:
df_g.head()

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,...,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,pickup_longitude,pickup_latitude
0,2,2018-06-01 00:33:55,2018-06-01 00:36:13,N,1,66,33,5,0.51,4.0,...,0.5,0.70,0.0,NaN,0.3,6.00,1,1,9.877850e+05,194926.521869
1,2,2018-06-01 00:40:36,2018-06-01 00:49:46,N,1,25,49,5,1.97,9.0,...,0.5,2.06,0.0,NaN,0.3,12.36,1,1,9.880019e+05,189102.798643
2,2,2018-06-01 00:57:12,2018-06-01 01:02:58,N,1,61,49,5,1.40,6.5,...,0.5,0.00,0.0,NaN,0.3,7.80,2,1,1.000223e+06,184828.371224
3,2,2018-06-01 00:10:13,2018-06-01 00:16:27,N,1,49,97,1,1.36,7.0,...,0.5,0.00,0.0,NaN,0.3,8.30,2,1,9.948706e+05,190214.741724
4,1,2018-06-01 00:32:08,2018-06-01 00:52:06,N,1,75,127,1,7.90,24.0,...,0.5,6.30,0.0,NaN,0.3,31.60,1,1,9.993649e+05,227274.374428


In [12]:
df_g['pickup_longitude'] = df_g['pickup_longitude'].apply(lambda x: '{:.2f}'.format(x))

In [13]:
df_g['pickup_latitude'] = df_g['pickup_latitude'].apply(lambda x: '{:.2f}'.format(x))

In [14]:
df_g.head()

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,...,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,pickup_longitude,pickup_latitude
0,2,2018-06-01 00:33:55,2018-06-01 00:36:13,N,1,66,33,5,0.51,4.0,...,0.5,0.70,0.0,NaN,0.3,6.00,1,1,987785.05,194926.52
1,2,2018-06-01 00:40:36,2018-06-01 00:49:46,N,1,25,49,5,1.97,9.0,...,0.5,2.06,0.0,NaN,0.3,12.36,1,1,988001.91,189102.80
2,2,2018-06-01 00:57:12,2018-06-01 01:02:58,N,1,61,49,5,1.40,6.5,...,0.5,0.00,0.0,NaN,0.3,7.80,2,1,1000222.66,184828.37
3,2,2018-06-01 00:10:13,2018-06-01 00:16:27,N,1,49,97,1,1.36,7.0,...,0.5,0.00,0.0,NaN,0.3,8.30,2,1,994870.57,190214.74
4,1,2018-06-01 00:32:08,2018-06-01 00:52:06,N,1,75,127,1,7.90,24.0,...,0.5,6.30,0.0,NaN,0.3,31.60,1,1,999364.85,227274.37


In [15]:
df_g = df_g.join(df_loc.set_index('LocationID'),on=['DOLocationID'])
df_g.drop(['OBJECTID', 'Shape_Leng', 'Shape_Area', 'zone', 'borough'], axis=1, inplace=True)
df_g.head()

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,...,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,pickup_longitude,pickup_latitude,longitude,latitude
0,2,2018-06-01 00:33:55,2018-06-01 00:36:13,N,1,66,33,5,0.51,4.0,...,0.0,NaN,0.3,6.00,1,1,987785.05,194926.52,9.853533e+05,192941.713096
1,2,2018-06-01 00:40:36,2018-06-01 00:49:46,N,1,25,49,5,1.97,9.0,...,0.0,NaN,0.3,12.36,1,1,988001.91,189102.80,9.948706e+05,190214.741724
2,2,2018-06-01 00:57:12,2018-06-01 01:02:58,N,1,61,49,5,1.40,6.5,...,0.0,NaN,0.3,7.80,2,1,1000222.66,184828.37,9.948706e+05,190214.741724
3,2,2018-06-01 00:10:13,2018-06-01 00:16:27,N,1,49,97,1,1.36,7.0,...,0.0,NaN,0.3,8.30,2,1,994870.57,190214.74,9.908516e+05,190708.984524
4,1,2018-06-01 00:32:08,2018-06-01 00:52:06,N,1,75,127,1,7.90,24.0,...,0.0,NaN,0.3,31.60,1,1,999364.85,227274.37,1.006493e+06,254234.219554


In [16]:
df_g.rename(columns={'longitude': 'dropoff_longitude', 'latitude': 'dropoff_latitude', 'lpep_pickup_datetime': 'pickup_datetime', 'lpep_dropoff_datetime':'dropoff_datetime'}, inplace=True)
df_g.head()

,VendorID,pickup_datetime,dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,...,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude
0,2,2018-06-01 00:33:55,2018-06-01 00:36:13,N,1,66,33,5,0.51,4.0,...,0.0,NaN,0.3,6.00,1,1,987785.05,194926.52,9.853533e+05,192941.713096
1,2,2018-06-01 00:40:36,2018-06-01 00:49:46,N,1,25,49,5,1.97,9.0,...,0.0,NaN,0.3,12.36,1,1,988001.91,189102.80,9.948706e+05,190214.741724
2,2,2018-06-01 00:57:12,2018-06-01 01:02:58,N,1,61,49,5,1.40,6.5,...,0.0,NaN,0.3,7.80,2,1,1000222.66,184828.37,9.948706e+05,190214.741724
3,2,2018-06-01 00:10:13,2018-06-01 00:16:27,N,1,49,97,1,1.36,7.0,...,0.0,NaN,0.3,8.30,2,1,994870.57,190214.74,9.908516e+05,190708.984524
4,1,2018-06-01 00:32:08,2018-06-01 00:52:06,N,1,75,127,1,7.90,24.0,...,0.0,NaN,0.3,31.60,1,1,999364.85,227274.37,1.006493e+06,254234.219554


In [17]:
df_g['dropoff_longitude'] = df_g['dropoff_longitude'].apply(lambda x: '{:.2f}'.format(x))
df_g['dropoff_latitude'] = df_g['dropoff_latitude'].apply(lambda x: '{:.2f}'.format(x))

In [18]:
df_g.head()

,VendorID,pickup_datetime,dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,...,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude
0,2,2018-06-01 00:33:55,2018-06-01 00:36:13,N,1,66,33,5,0.51,4.0,...,0.0,NaN,0.3,6.00,1,1,987785.05,194926.52,985353.26,192941.71
1,2,2018-06-01 00:40:36,2018-06-01 00:49:46,N,1,25,49,5,1.97,9.0,...,0.0,NaN,0.3,12.36,1,1,988001.91,189102.80,994870.57,190214.74
2,2,2018-06-01 00:57:12,2018-06-01 01:02:58,N,1,61,49,5,1.40,6.5,...,0.0,NaN,0.3,7.80,2,1,1000222.66,184828.37,994870.57,190214.74
3,2,2018-06-01 00:10:13,2018-06-01 00:16:27,N,1,49,97,1,1.36,7.0,...,0.0,NaN,0.3,8.30,2,1,994870.57,190214.74,990851.61,190708.98
4,1,2018-06-01 00:32:08,2018-06-01 00:52:06,N,1,75,127,1,7.90,24.0,...,0.0,NaN,0.3,31.60,1,1,999364.85,227274.37,1006493.44,254234.22


In [19]:
df_g.isnull().sum()

VendorID                      0
pickup_datetime               0
dropoff_datetime              0
store_and_fwd_flag            0
RatecodeID                    0
PULocationID                  0
DOLocationID                  0
passenger_count               0
trip_distance                 0
fare_amount                   0
extra                         0
mta_tax                       0
tip_amount                    0
tolls_amount                  0
ehail_fee                763022
improvement_surcharge         0
total_amount                  0
payment_type                  0
trip_type                     0
pickup_longitude              0
pickup_latitude               0
dropoff_longitude             0
dropoff_latitude              0
dtype: int64

In [20]:
df_g.count()

VendorID                 763022
pickup_datetime          763022
dropoff_datetime         763022
store_and_fwd_flag       763022
RatecodeID               763022
PULocationID             763022
DOLocationID             763022
passenger_count          763022
trip_distance            763022
fare_amount              763022
extra                    763022
mta_tax                  763022
tip_amount               763022
tolls_amount             763022
ehail_fee                     0
improvement_surcharge    763022
total_amount             763022
payment_type             763022
trip_type                763022
pickup_longitude         763022
pickup_latitude          763022
dropoff_longitude        763022
dropoff_latitude         763022
dtype: int64

In [21]:
df_g.dropna(subset=['pickup_datetime','dropoff_datetime','pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude'])

,VendorID,pickup_datetime,dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,...,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude
0,2,2018-06-01 00:33:55,2018-06-01 00:36:13,N,1,66,33,5,0.51,4.0,...,0.0,NaN,0.3,6.00,1,1,987785.05,194926.52,985353.26,192941.71
1,2,2018-06-01 00:40:36,2018-06-01 00:49:46,N,1,25,49,5,1.97,9.0,...,0.0,NaN,0.3,12.36,1,1,988001.91,189102.80,994870.57,190214.74
2,2,2018-06-01 00:57:12,2018-06-01 01:02:58,N,1,61,49,5,1.40,6.5,...,0.0,NaN,0.3,7.80,2,1,1000222.66,184828.37,994870.57,190214.74
3,2,2018-06-01 00:10:13,2018-06-01 00:16:27,N,1,49,97,1,1.36,7.0,...,0.0,NaN,0.3,8.30,2,1,994870.57,190214.74,990851.61,190708.98
4,1,2018-06-01 00:32:08,2018-06-01 00:52:06,N,1,75,127,1,7.90,24.0,...,0.0,NaN,0.3,31.60,1,1,999364.85,227274.37,1006493.44,254234.22
5,1,2018-06-01 00:15:35,2018-06-01 00:27:44,N,1,36,112,2,2.90,11.0,...,0.0,NaN,0.3,14.30,1,1,1007105.37,193942.63,998612.72,204798.78
6,1,2018-06-01 00:49:03,2018-06-01 00:57:26,N,1,256,49,1,2.60,9.5,...,0.0,NaN,0.3,12.95,1,1,995524.65,198307.93,994870.57,190214.74
7,2,2018-06-01 00:49:31,2018-06-01 01:38:02,N,1,25,28,2,14.64,45.0,...,0.0,NaN,0.3,46.30,1,1,988001.91,189102.80,1037383.34,198281.30
8,1,2018-06-01 00:16:57,2018-06-01 00:22:05,N,1,179,179,1,0.90,5.5,...,0.0,NaN,0.3,8.80,1,1,1004253.56,220334.19,1004253.56,220334.19
9,1,2018-06-01 00:42:02,2018-06-01 00:46:15,N,1,179,179,2,0.90,5.5,...,0.0,NaN,0.3,8.80,1,1,1004253.56,220334.19,1004253.56,220334.19


In [8]:
#YELLOW_TAXI_DATA_2018_JUNE

df_y = pd.read_csv("/Users/ranjani/taxidata/yellow_tripdata_2018-06.csv")
df_y.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,1,2018-06-01 00:15:40,2018-06-01 00:16:46,1,0.0,1,N,145,145,2,3.0,0.5,0.5,0.00,0.0,0.3,4.30
1,1,2018-06-01 00:04:18,2018-06-01 00:09:18,1,1.0,1,N,230,161,1,5.5,0.5,0.5,1.35,0.0,0.3,8.15
2,1,2018-06-01 00:14:39,2018-06-01 00:29:46,1,3.3,1,N,100,263,2,13.0,0.5,0.5,0.00,0.0,0.3,14.30
3,1,2018-06-01 00:51:25,2018-06-01 00:51:29,3,0.0,1,N,145,145,2,2.5,0.5,0.5,0.00,0.0,0.3,3.80
4,1,2018-06-01 00:55:06,2018-06-01 00:55:10,1,0.0,1,N,145,145,2,2.5,0.5,0.5,0.00,0.0,0.3,3.80


In [9]:
df_y = df_y.join(df_loc.set_index('LocationID'),on=['PULocationID'])
df_y.drop(['OBJECTID', 'Shape_Leng', 'Shape_Area', 'zone', 'borough'], axis=1, inplace=True)
df_y.rename(columns={'longitude': 'pickup_longitude', 'latitude': 'pickup_latitude'}, inplace=True)

NameError: name 'df_loc' is not defined

In [10]:
df_y.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,1,2018-06-01 00:15:40,2018-06-01 00:16:46,1,0.0,1,N,145,145,2,3.0,0.5,0.5,0.00,0.0,0.3,4.30
1,1,2018-06-01 00:04:18,2018-06-01 00:09:18,1,1.0,1,N,230,161,1,5.5,0.5,0.5,1.35,0.0,0.3,8.15
2,1,2018-06-01 00:14:39,2018-06-01 00:29:46,1,3.3,1,N,100,263,2,13.0,0.5,0.5,0.00,0.0,0.3,14.30
3,1,2018-06-01 00:51:25,2018-06-01 00:51:29,3,0.0,1,N,145,145,2,2.5,0.5,0.5,0.00,0.0,0.3,3.80
4,1,2018-06-01 00:55:06,2018-06-01 00:55:10,1,0.0,1,N,145,145,2,2.5,0.5,0.5,0.00,0.0,0.3,3.80


In [11]:
df_y['passenger_count'].isnull().sum()

0

In [25]:
df_y['pickup_longitude'] = df_y['pickup_longitude'].apply(lambda x: '{:.2f}'.format(x))
df_y['pickup_latitude'] = df_y['pickup_latitude'].apply(lambda x: '{:.2f}'.format(x))

In [26]:
df_y.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,pickup_longitude,pickup_latitude
0,1,2018-06-01 00:15:40,2018-06-01 00:16:46,1,0.0,1,N,145,145,2,3.0,0.5,0.5,0.00,0.0,0.3,4.30,998207.57,211280.69
1,1,2018-06-01 00:04:18,2018-06-01 00:09:18,1,1.0,1,N,230,161,1,5.5,0.5,0.5,1.35,0.0,0.3,8.15,988628.08,216098.91
2,1,2018-06-01 00:14:39,2018-06-01 00:29:46,1,3.3,1,N,100,263,2,13.0,0.5,0.5,0.00,0.0,0.3,14.30,987355.56,213800.92
3,1,2018-06-01 00:51:25,2018-06-01 00:51:29,3,0.0,1,N,145,145,2,2.5,0.5,0.5,0.00,0.0,0.3,3.80,998207.57,211280.69
4,1,2018-06-01 00:55:06,2018-06-01 00:55:10,1,0.0,1,N,145,145,2,2.5,0.5,0.5,0.00,0.0,0.3,3.80,998207.57,211280.69


In [27]:
df_y = df_y.join(df_loc.set_index('LocationID'),on=['DOLocationID'])
df_y.drop(['OBJECTID', 'Shape_Leng', 'Shape_Area', 'zone', 'borough'], axis=1, inplace=True)
df_y.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,...,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,pickup_longitude,pickup_latitude,longitude,latitude
0,1,2018-06-01 00:15:40,2018-06-01 00:16:46,1,0.0,1,N,145,145,2,...,0.5,0.5,0.00,0.0,0.3,4.30,998207.57,211280.69,998207.572970,211280.687978
1,1,2018-06-01 00:04:18,2018-06-01 00:09:18,1,1.0,1,N,230,161,1,...,0.5,0.5,1.35,0.0,0.3,8.15,988628.08,216098.91,990433.407940,215448.355221
2,1,2018-06-01 00:14:39,2018-06-01 00:29:46,1,3.3,1,N,100,263,2,...,0.5,0.5,0.00,0.0,0.3,14.30,987355.56,213800.92,997818.271491,223006.185937
3,1,2018-06-01 00:51:25,2018-06-01 00:51:29,3,0.0,1,N,145,145,2,...,0.5,0.5,0.00,0.0,0.3,3.80,998207.57,211280.69,998207.572970,211280.687978
4,1,2018-06-01 00:55:06,2018-06-01 00:55:10,1,0.0,1,N,145,145,2,...,0.5,0.5,0.00,0.0,0.3,3.80,998207.57,211280.69,998207.572970,211280.687978


In [28]:
df_y.rename(columns={'longitude': 'dropoff_longitude', 'latitude': 'dropoff_latitude', 'tpep_pickup_datetime': 'pickup_datetime', 'tpep_dropoff_datetime':'dropoff_datetime'}, inplace=True)

In [29]:
df_y.head()

,VendorID,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,...,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude
0,1,2018-06-01 00:15:40,2018-06-01 00:16:46,1,0.0,1,N,145,145,2,...,0.5,0.5,0.00,0.0,0.3,4.30,998207.57,211280.69,998207.572970,211280.687978
1,1,2018-06-01 00:04:18,2018-06-01 00:09:18,1,1.0,1,N,230,161,1,...,0.5,0.5,1.35,0.0,0.3,8.15,988628.08,216098.91,990433.407940,215448.355221
2,1,2018-06-01 00:14:39,2018-06-01 00:29:46,1,3.3,1,N,100,263,2,...,0.5,0.5,0.00,0.0,0.3,14.30,987355.56,213800.92,997818.271491,223006.185937
3,1,2018-06-01 00:51:25,2018-06-01 00:51:29,3,0.0,1,N,145,145,2,...,0.5,0.5,0.00,0.0,0.3,3.80,998207.57,211280.69,998207.572970,211280.687978
4,1,2018-06-01 00:55:06,2018-06-01 00:55:10,1,0.0,1,N,145,145,2,...,0.5,0.5,0.00,0.0,0.3,3.80,998207.57,211280.69,998207.572970,211280.687978


In [30]:
df_y['dropoff_longitude'] = df_y['dropoff_longitude'].apply(lambda x: '{:.2f}'.format(x))
df_y['dropoff_latitude'] = df_y['dropoff_latitude'].apply(lambda x: '{:.2f}'.format(x))

In [31]:
df_y.head()

,VendorID,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,...,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude
0,1,2018-06-01 00:15:40,2018-06-01 00:16:46,1,0.0,1,N,145,145,2,...,0.5,0.5,0.00,0.0,0.3,4.30,998207.57,211280.69,998207.57,211280.69
1,1,2018-06-01 00:04:18,2018-06-01 00:09:18,1,1.0,1,N,230,161,1,...,0.5,0.5,1.35,0.0,0.3,8.15,988628.08,216098.91,990433.41,215448.36
2,1,2018-06-01 00:14:39,2018-06-01 00:29:46,1,3.3,1,N,100,263,2,...,0.5,0.5,0.00,0.0,0.3,14.30,987355.56,213800.92,997818.27,223006.19
3,1,2018-06-01 00:51:25,2018-06-01 00:51:29,3,0.0,1,N,145,145,2,...,0.5,0.5,0.00,0.0,0.3,3.80,998207.57,211280.69,998207.57,211280.69
4,1,2018-06-01 00:55:06,2018-06-01 00:55:10,1,0.0,1,N,145,145,2,...,0.5,0.5,0.00,0.0,0.3,3.80,998207.57,211280.69,998207.57,211280.69


In [32]:
df_y.isnull().sum()

VendorID                 0
pickup_datetime          0
dropoff_datetime         0
passenger_count          0
trip_distance            0
RatecodeID               0
store_and_fwd_flag       0
PULocationID             0
DOLocationID             0
payment_type             0
fare_amount              0
extra                    0
mta_tax                  0
tip_amount               0
tolls_amount             0
improvement_surcharge    0
total_amount             0
pickup_longitude         0
pickup_latitude          0
dropoff_longitude        0
dropoff_latitude         0
dtype: int64

In [33]:
df_y.count()

VendorID                 8722693
pickup_datetime          8722693
dropoff_datetime         8722693
passenger_count          8722693
trip_distance            8722693
RatecodeID               8722693
store_and_fwd_flag       8722693
PULocationID             8722693
DOLocationID             8722693
payment_type             8722693
fare_amount              8722693
extra                    8722693
mta_tax                  8722693
tip_amount               8722693
tolls_amount             8722693
improvement_surcharge    8722693
total_amount             8722693
pickup_longitude         8722693
pickup_latitude          8722693
dropoff_longitude        8722693
dropoff_latitude         8722693
dtype: int64

In [34]:
df_y.dropna(subset=['pickup_datetime','dropoff_datetime','pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude'])

,VendorID,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,...,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude
0,1,2018-06-01 00:15:40,2018-06-01 00:16:46,1,0.00,1,N,145,145,2,...,0.5,0.5,0.00,0.0,0.3,4.30,998207.57,211280.69,998207.57,211280.69
1,1,2018-06-01 00:04:18,2018-06-01 00:09:18,1,1.00,1,N,230,161,1,...,0.5,0.5,1.35,0.0,0.3,8.15,988628.08,216098.91,990433.41,215448.36
2,1,2018-06-01 00:14:39,2018-06-01 00:29:46,1,3.30,1,N,100,263,2,...,0.5,0.5,0.00,0.0,0.3,14.30,987355.56,213800.92,997818.27,223006.19
3,1,2018-06-01 00:51:25,2018-06-01 00:51:29,3,0.00,1,N,145,145,2,...,0.5,0.5,0.00,0.0,0.3,3.80,998207.57,211280.69,998207.57,211280.69
4,1,2018-06-01 00:55:06,2018-06-01 00:55:10,1,0.00,1,N,145,145,2,...,0.5,0.5,0.00,0.0,0.3,3.80,998207.57,211280.69,998207.57,211280.69
5,1,2018-06-01 00:09:00,2018-06-01 00:24:01,1,2.00,1,N,161,234,1,...,0.5,0.5,2.55,0.0,0.3,15.35,990433.41,215448.36,986894.30,208964.96
6,1,2018-06-01 00:02:33,2018-06-01 00:13:01,2,1.50,1,N,163,233,1,...,0.5,0.5,1.95,0.0,0.3,11.75,990609.94,217764.31,992568.05,212203.98
7,1,2018-06-01 00:13:23,2018-06-01 00:16:52,1,0.70,1,N,186,246,1,...,0.5,0.5,1.85,0.0,0.3,8.15,986346.34,211974.16,983193.73,213400.59
8,1,2018-06-01 00:24:29,2018-06-01 01:08:43,1,5.70,1,N,230,179,2,...,0.5,0.5,0.00,0.0,0.3,23.30,988628.08,216098.91,1004253.56,220334.19
9,2,2018-06-01 00:17:01,2018-06-01 00:23:16,1,0.85,1,N,179,223,2,...,0.5,0.5,0.00,0.0,0.3,7.30,1004253.56,220334.19,1010330.24,222885.45


In [35]:
df_g.to_csv("green_tripdata_2018_june_output.csv", index=False)

In [36]:
df_y.to_csv("yellow_tripdata_2018_june_output.csv", index=False)